El paquete transformers de Hugging Face utiliza modelos preentrenados de BERT para realizar tareas de NLP

https://huggingface.co/transformers/quickstart.html


In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import torch

from transformers import AutoTokenizer, AutoModelForQuestionAnswering


Se descarga el modelo preentrenado, esto descarga los pesos de un modelo preentrenado con BERT.

(La primera vez que se corre baja un modelo de 13GB)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

Cargo los datos ya convertidos a dataframe

In [18]:
%%time
df_path = 'D:\DS\COVID\outputs'

df = pd.read_csv(os.path.join(df_path, 'result.csv'), 
                 dtype={'title_x': str, 'abstract_x': str, 'body_text': str, 'has_full_text': str
                       }
                )

Wall time: 17.6 s


In [ ]:
texts = df[df['abstract_y'].notnull()]['abstract_y'].tail(10) # Pruebo con pocos textos

Creo un conjunto de preguntas

In [44]:
questions = [
    "What strategies are effective to prevent secondary transmission?",
    "What is the incubation period?",
    "What is the incubation period across age and health status?",
    "How long individuals are contagious?"
]


Busco las preguntas en los textos.

Este proceso devuelve dos tokens, que son los que tienen mayor probabilidad de ser el comienzo y el fin de la respuesta.  Luego, la respuesta estará entre estos dos tokens.

In [43]:
for question in questions:
    for text in texts:
        inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt",
                                       max_length=511, truncation_strategy='longest_first')
        input_ids = inputs["input_ids"].tolist()[0]

        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        answer_start_scores, answer_end_scores = model(**inputs)

        answer_start = torch.argmax(
            answer_start_scores
        )  # Obtiene el token con mayor probabilidad de ser comienzo de respuesta
        answer_end = torch.argmax(answer_end_scores) + 1  # Obtiene el token con mayor probabilidad de ser fin de respuesta

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        print(f"Question: {question}")
        print(f"Answer: {answer}\n")

Question: What strategies are effective to prevent secondary transmission?
Answer: hospitalisation and isolation

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: surface disinfection

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What strategies are effective to prevent secondary transmission?
Answer: [CLS]

Question: What are effective movement control strategies?
Ans